In [2]:
import random
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from dataclasses import dataclass

In [3]:
seed = 42

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed);

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device {device}')

Using device cuda


### Prepare the data

In [5]:
class ImpurityDataset(Dataset):
    
    def __init__(self, dataframe, fixed_features, labels, feature_scaler=None, label_scaler=None, device=None):
        assert len(labels) % 2 == 0
        
        self.fixed_features = fixed_features
        self.labels = labels
        self.n_samples = len(dataframe)
        
        self.output_length = len(labels)
        self.input_length = len(fixed_features)
        self.sequence_length = 1
        
        df_features = dataframe[fixed_features]
        df_labels = dataframe[labels]

        if feature_scaler is not None and label_scaler is not None:
            xs = feature_scaler.transform(df_features.values)
            ys = label_scaler.transform(df_labels.values)
        else:
            xs = df_features.values
            ys = df_labels.values

        feature_data = np.zeros((self.n_samples, self.sequence_length, self.input_length))
        label_data = np.zeros((self.n_samples, self.sequence_length, self.output_length))
        
        for i in range(self.n_samples):
            xi = xs[i]
            yi = ys[i]
            
            feature_data[i, :, :] = xi
            label_data[i, :, :] = yi

        self.feature_data = torch.tensor(feature_data, dtype=torch.float).to(device)
        self.label_data = torch.tensor(label_data, dtype=torch.float).to(device)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        return self.feature_data[idx], self.label_data[idx]

In [6]:
def compute_scalers(dataframe, fixed_features, labels, test_size, random_state=None):
    train_df, _ = train_test_split(dataframe, test_size=test_size, random_state=random_state)
    df_features = train_df[fixed_features]
    df_labels = train_df[labels]
    
    feature_scaler = StandardScaler()
    feature_scaler.fit(df_features.values)
    
    label_scaler = StandardScaler()
    label_scaler.fit(df_labels.values)

    return feature_scaler, label_scaler      

In [ ]:
file_path = '../data/20230825_144318_10k_EVDoubExp-TExp-wmax5-sparse-hyb_with_perturbation.csv'

#fixed_features = ['beta', 'U', 'Eimp', 'E1', 'E2', 'E3', 'V1', 'V2', 'V3']

# original feature set
#fixed_features = ['beta', 'E1', 'E2', 'E3', 'V1', 'V2', 'V3']

fixed_features = ['beta', 'E1', 'E2', 'E3', 'V1', 'V2', 'V3', 'ReFso1', 'ImFso1', 'ReFso3', 'ImFso3', 'ReFso5', 'ImFso5', 'ReFso7', 'ImFso7', 'ReFso9', 'ImFso9', 'ReFso11', 'ImFso11', 'ReFso13', 'ImFso13', 'ReFso15', 'ImFso15', 'ReFso17', 'ImFso17', 'ReFso19', 'ImFso19', 'ReFso21', 'ImFso21', 'ReFso23', 'ImFso23', 'ReFso25', 'ImFso25', 'ReFso29', 'ImFso29', 'ReFso33', 'ImFso33', 'ReFso37', 'ImFso37', 'ReFso43', 'ImFso43', 'ReFso49', 'ImFso49', 'ReFso57', 'ImFso57', 'ReFso69', 'ImFso69', 'ReFso83', 'ImFso83', 'ReFso101', 'ImFso101', 'ReFso127', 'ImFso127', 'ReFso165', 'ImFso165', 'ReFso237', 'ImFso237', 'ReFso399', 'ImFso399', 'ReFso1207', 'ImFso1207']

labels = ['ReSf1', 'ImSf1', 'ReSf3', 'ImSf3', 'ReSf5', 'ImSf5', 'ReSf7', 'ImSf7', 'ReSf9', 'ImSf9', 'ReSf11', 'ImSf11', 'ReSf13', 'ImSf13', 'ReSf15', 'ImSf15', 'ReSf17', 'ImSf17', 'ReSf19', 'ImSf19', 'ReSf21', 'ImSf21', 'ReSf23', 'ImSf23', 'ReSf25', 'ImSf25', 'ReSf29', 'ImSf29', 'ReSf33', 'ImSf33', 'ReSf37', 'ImSf37', 'ReSf43', 'ImSf43', 'ReSf49', 'ImSf49', 'ReSf57', 'ImSf57', 'ReSf69', 'ImSf69', 'ReSf83', 'ImSf83', 'ReSf101', 'ImSf101', 'ReSf127', 'ImSf127', 'ReSf165', 'ImSf165', 'ReSf237', 'ImSf237', 'ReSf399', 'ImSf399', 'ReSf1207', 'ImSf1207']

df = pd.read_csv(file_path, skiprows=4) # we skip the first four lines, because they are just metadata
df = df[fixed_features + labels]

# remove one special row, looks very weird; ReSf1 = 2.377167465976437e-06
df = df[df['ReSf1'] >= 1e-05]

validation_size = 0.1 # 90% training, 10% for validation

use_scaling = True

if use_scaling:
    feature_scaler, label_scaler = compute_scalers(df, fixed_features, labels, validation_size, seed) # make sure we use the same seed, otherwise the two splits differ!
    dataset = ImpurityDataset(df, fixed_features, labels, feature_scaler, label_scaler, device=device)
else:
    dataset = ImpurityDataset(df, fixed_features, labels, device=device)

indices = list(range(len(dataset)))
train_indices, val_indices = train_test_split(indices, test_size=validation_size, random_state=seed)  # make sure we use the same seed, otherwise the two splits differ!

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=0)

In [7]:
dataset.__getitem__(0);

In [8]:
sorted(df['ReSf1'].abs())[:10];

### Define the model

In [9]:
@dataclass
class ModelConfig:
    input_dim: int
    output_dim: int
    sequence_length: int
    
    d_model: int
    nhead: int
    num_layers: int
    dim_feedforward: int
    
    dropout: float
    activation: str
    bias: bool

class AutoregressiveTransformer(nn.Module):
    
    def __init__(self, config, device):
        super(AutoregressiveTransformer, self).__init__()

        self.config = config
        self.input_projection = nn.Linear(config.input_dim, config.d_model)
        
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=config.d_model, 
            nhead=config.nhead, 
            dim_feedforward=config.dim_feedforward, 
            dropout=config.dropout,
            activation=config.activation, 
            batch_first=True, 
            norm_first=True, 
            bias=config.bias
        )
        
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=config.num_layers)
        self.output_layer = nn.Linear(config.d_model, config.output_dim)
        
    def forward(self, x):        
        x = self.input_projection(x)
        output = self.transformer_decoder(x, x)
        output = self.output_layer(output)
        
        return output

### Initialize the model

In [1]:
config = ModelConfig(
    input_dim = len(fixed_features),
    output_dim = len(labels),
    sequence_length = 1,
    
    d_model = 128,
    nhead = 4,
    num_layers = 4,
    dim_feedforward = 128 * 4,
    
    dropout = 0.2,
    activation = 'gelu',
    bias = True
)

model = AutoregressiveTransformer(config, device).to(device)
print(sum(p.numel() for p in model.parameters())/1e3, 'k parameters')
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

NameError: name 'ModelConfig' is not defined

In [26]:
def train(model, loader, optimizer, criterion, device):
    model.train()
    losses = []
    
    for inputs, targets in loader:
        
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())

    return np.average(losses)

In [27]:
def validate(model, loader, criterion, device):
    model.eval()
    losses = []
    
    with torch.no_grad():
        
        for inputs, targets in loader:
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            losses.append(loss.item())
            
    return np.average(losses)

In [28]:
def validate_mape(model, loader, device, use_scaling=use_scaling, epsilon=1e-8):
    model.eval()
    losses = []
    
    with torch.no_grad():
        
        for inputs, targets in loader:
            outputs = model(inputs)
            
            if use_scaling:
                outputs = torch.tensor(reverse_tranform_output(outputs, label_scaler))
                targets = torch.tensor(reverse_tranform_output(targets, label_scaler))
            
            ape = torch.abs((targets - outputs) / (targets + epsilon))
            mape = torch.mean(ape) * 100
            
            losses.append(mape.item())

    return np.average(losses)

In [29]:
def reverse_tranform_output(data, scaler):
    B, T, C = data.shape
    data = data.view(B, T*C)
    data = data.cpu().numpy()
    return scaler.inverse_transform(data)

In [30]:
masked_output = len(fixed_features) - 1

validate_mape(model, val_loader, device, use_scaling=use_scaling)

102.34923112392426

### Train the model

In [31]:
num_epochs = 1000

# Initialize variables to store the lowest losses and corresponding epochs
lowest_train_loss = float('inf')
lowest_val_loss = float('inf')
best_train_epoch = -1
best_val_epoch = -1

for epoch in range(num_epochs):
    train(model, train_loader, optimizer, criterion, device)
    
    train_loss = validate(model, train_loader, criterion, device)
    val_loss = validate(model, val_loader, criterion, device)
    
    train_mape = validate_mape(model, train_loader, device, use_scaling=use_scaling)
    val_mape = validate_mape(model, val_loader, device, use_scaling=use_scaling)

    print(f"Epoch {(epoch+1):3d}: Train Loss: {train_loss:.6f}, Train MAPE: {train_mape:.3f}%, Val Loss: {val_loss:.6f}, Val MAPE: {val_mape:.3f}%")
    
    if train_mape < lowest_train_loss:
        lowest_train_loss = train_mape
        best_train_epoch = epoch + 1

    if val_mape < lowest_val_loss:
        lowest_val_loss = val_mape
        best_val_epoch = epoch + 1

print(f"\nLowest Train Loss: {lowest_train_loss:.6f} at Epoch {best_train_epoch}")
print(f"Lowest Val Loss: {lowest_val_loss:.6f} at Epoch {best_val_epoch}")


Epoch   1: Train Loss: 0.175977, Train MAPE: 32.318%, Val Loss: 0.176459, Val MAPE: 39.739%
Epoch   2: Train Loss: 0.118432, Train MAPE: 24.353%, Val Loss: 0.117937, Val MAPE: 28.867%
Epoch   3: Train Loss: 0.093050, Train MAPE: 20.829%, Val Loss: 0.101579, Val MAPE: 24.693%
Epoch   4: Train Loss: 0.075428, Train MAPE: 19.087%, Val Loss: 0.080443, Val MAPE: 21.833%
Epoch   5: Train Loss: 0.063596, Train MAPE: 17.024%, Val Loss: 0.068555, Val MAPE: 19.401%
Epoch   6: Train Loss: 0.052500, Train MAPE: 14.053%, Val Loss: 0.057614, Val MAPE: 16.574%
Epoch   7: Train Loss: 0.044508, Train MAPE: 13.064%, Val Loss: 0.054206, Val MAPE: 15.702%
Epoch   8: Train Loss: 0.039735, Train MAPE: 11.894%, Val Loss: 0.049478, Val MAPE: 14.684%
Epoch   9: Train Loss: 0.034581, Train MAPE: 11.025%, Val Loss: 0.045655, Val MAPE: 13.061%
Epoch  10: Train Loss: 0.036524, Train MAPE: 11.390%, Val Loss: 0.046897, Val MAPE: 14.210%
Epoch  11: Train Loss: 0.032500, Train MAPE: 10.164%, Val Loss: 0.042416, Val MA

### Sample from the model

In [182]:
def sample_from_model(model, input, sequence_length, fixed_features_len, device):
    model.eval()

    input = input[:fixed_features_len, :]
    T, C = input.shape
    input = input.view(1, T, C)
    outputs = torch.zeros(sequence_length, 2, device=device)
    
    with torch.no_grad():
    
        for i in range(0, sequence_length):
            output = model(input)
            predictions = output[:, -1, :].view(1, 1, 2)
            outputs[i, :] = predictions
            
            if i == sequence_length:
                break

            input = torch.cat((input, predictions), dim=1)

    return outputs

In [183]:
def calculate_mape(true_values, predictions, epsilon=1e-8):
    mape = torch.mean(torch.abs((true_values - predictions) / (true_values + epsilon))) * 100
    return mape.item()

In [184]:
def autoregressive_error(model, loader, n_samples, sequence_length, fixed_features_len, use_scaling):

    mapes = []
    
    for idx in range(n_samples):
        input = loader.dataset[idx][0]
        
        outputs = sample_from_model(model, input, sequence_length, fixed_features_len, device)
        targets = loader.dataset[idx][1][fixed_features_len-1:, :]

        T, C = outputs.shape
        outputs = outputs.view(1, T, C)
        targets = targets.view(1, T, C)
        
        if use_scaling:
            outputs = torch.tensor(reverse_tranform_output(outputs, 0, label_scaler))
            targets = torch.tensor(reverse_tranform_output(targets, 0, label_scaler))
        
        mapes.append(calculate_mape(targets, outputs))

    return np.mean(mapes)

In [185]:
n_samples = 100
sequence_length = 27
fixed_features_len = len(fixed_features)
loader = train_loader

autoregressive_error(model, loader, n_samples, sequence_length, fixed_features_len, use_scaling)

RuntimeError: shape '[1, 1, 2]' is invalid for input of size 54

In [ ]:
n_samples = 1000
sequence_length = 27
fixed_features_len = len(fixed_features)
max_features = fixed_features_len + len(labels) - 2

total_mapes = []

for j in range(sequence_length):
    fixed_labels_len = j
    
    mapes = []
    
    for idx in range(n_samples):
        input = val_loader.dataset[idx][0][fixed_labels_len]
        output = sample_from_model(model, input, sequence_length, fixed_features_len, fixed_labels_len, max_features, device)
        
        target = convert(val_loader.dataset[idx][1].reshape(-1))
        output = convert(output.reshape(-1))
        
        mapes.append(calculate_mape(target, output))
    
        if (idx+1) % 1000 == 0:
            print(f"sequence {idx/1000} done")

    total_mapes.append(np.mean(mapes))

print(total_mapes)